In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datasets import load_dataset
from tqdm import tqdm

M:\disco M\Python\venvs\env_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [50]:
import torch
from torchinfo import summary
from utils.text_generation import LMPipeline
from utils.transformer_decoder import DecoderLM
from utils.tokenizer import MyTokenizer

In [4]:
# download dataset
dataset = load_dataset("AiresPucrs/google-play-apps-review-pt")

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model Configs

In [53]:
VOCAB_SIZE = 12_000
MAX_LEN = 128
D_MODEL = 124
N_LAYERS = 6
N_HEADS = 4
HIDDEN_SIZE = 512
DROPOUT = 0.1
BATCH_SIZE = 8
EPOCHS = 50

# Tokenizer

In [6]:
tokenizer = MyTokenizer()

In [7]:
tokenizer.create_tokenizer(dataset)

In [8]:
# print(tokenizer.vocab_transform.get_itos())

In [9]:
print(f'Tokeniznado: {tokenizer.tokenize_text("ola")}')
print(f'Detokenizando: {tokenizer.untokenize_tokens([2 ,857, 3])}')

Tokeniznado: tensor([  2, 857,   3])
Detokenizando:  ola 


In [10]:
VOCAB_SIZE = len(tokenizer.vocab_transform)
VOCAB_SIZE

11465

# Dataset

In [11]:
from torch.utils.data import Dataset, DataLoader

In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 20000
    })
})

In [13]:
dataset['train'][0]['review']

'o aplicativo e bom disparadamente melhor que o concorrente whatsapp pontos positivos  possibilidade de aplicar temas personalizados para sair da aparencia padrao de acordo com o usuario  a nao utilizacao de um backup local e sem a possibilidade de perder todas as mensagens acidentalmente por ser um servico via nuvem  a possibilidade de usar bots como um diferencial alem de somente usar o aplicativo para conversar ou seja e possivel ampliar o uso do aplicativo para outras coisas interessantes como por exemplo estudar  a possibilidade de se entreter com jogos e se divertir com outros contatosamigos similar ao ponto anterior  a existencia de um chat secreto para autodestruir mensagens que 2 usuarios nao queiram que fiquem armazenadas na nuvem sendo assim uma forma de conversar com privacidade total ainda ha outros pontos positivos mas nao e necessario citar todos eu tenho somente um ponto negativo tal ponto e a instabilidade do sistema em nuvem do telegram que certas vezes dessincroniza 

In [16]:
class GoogleDataset(Dataset):
    def __init__(
        self,
        dataset,
        tokenizer: MyTokenizer,
        max_len: int = MAX_LEN,
    ):
        super().__init__()
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.vocab_size = len(self.tokenizer.vocab_transform)

    def __len__(self):
        return self.dataset['train'].num_rows

    def __getitem__(self, index):
        text = self.dataset['train'][index]['review']
        tokenized = self.tokenizer.tokenize_text(text)
        if len(tokenized) < self.max_len + 1:
            tokenized = tokenized.tolist()
            # porque colocar o <pad> antes do texto?
            tokenized = [self.tokenizer.PAD_IDX] * ((self.max_len + 1) - len(tokenized)) + tokenized
            tokenized = torch.tensor(tokenized)
        else:
            tokenized = tokenized[:self.max_len + 1]

        decoder_input = tokenized[: self.max_len]
        true_output = tokenized[1 : self.max_len + 1]
        # return decoder_input, true_output
        return {'decoder_input': decoder_input, 'true_output': true_output}

In [17]:
train_dataset = GoogleDataset(dataset, tokenizer)

In [18]:
# print(f'O que vai entrar no modelo: \n{train_dataset[12][0]}')
print(f'O que vai entrar no modelo: \n{train_dataset[12]["decoder_input"]}')
print('*'*30)
# print(f'O que deve sair do modelo: \n{train_dataset[12][1]}')
print(f'O que deve sair do modelo: \n{train_dataset[12]["true_output"]}')

O que vai entrar no modelo: 
tensor([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    2,   26,  528,   68,   21,  622,   19,  200,
           8,  192,  171,    5,   19,   10,   44,  622,    5,   10, 1017,    8,
         677,  428,  317, 1185,    7, 1167,   28,  777,    7,  225,   10,  256,
        1167,    5,   10,    8,   34,    5, 6690,   40, 1167,   20,  689,   80,
         385,  428,   12,  699,  216,    7, 1167, 1167,   20, 1244,  412,  216,
          10,  135,    8,    5, 4148,   30, 1483,   13, 9495, 2528,    6, 1547,
          43,   10,  260,   20, 2556,    5,  363, 1364,   10, 1206,   13,  129,
         564,    8,   58,  221, 1972, 7307, 1989,   15,   10, 1217,  203, 1757,
          12, 1525,   18, 4297,    5, 1986,  339, 2493])
******************************
O que deve sair do modelo: 
tensor([   0,    0,    0,    0,    0,    0,    0,    0,

# Model

In [34]:
import torch.nn as nn
from utils.transformer import TransformerDecoder


class DecoderLM(nn.Module):
    def __init__(
        self,
        vocab_size=VOCAB_SIZE,
        max_len=MAX_LEN,
        embed_dim=D_MODEL,
        num_layers=N_LAYERS,
        num_heads=N_HEADS,
        hidden_size=HIDDEN_SIZE,
        dropout=DROPOUT,
        pad_token_id=tokenizer.PAD_IDX,
    ):
        super().__init__()

        self.pad_token_id = pad_token_id
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

        self.decoder = TransformerDecoder(
            vocab_size=vocab_size,
            max_len=max_len,
            embed_dim=embed_dim,
            num_layers=num_layers,
            num_heads=num_heads,
            hidden_size=hidden_size,
            dropout=dropout,
        )

        self.lm_head = nn.Linear(embed_dim, vocab_size)

    def forward(self, x, non_pad_indexes=None):
        # we dont want to compute loss for padding tokens
        # get all hidden states
        logits = self.lm_head(self.decoder(x))
        # remove batch dimension
        logits = torch.reshape(logits, (-1, self.vocab_size))
        # get only the tokens that matter
        if non_pad_indexes is not None:
            logits = logits[non_pad_indexes, :]

        return logits

In [35]:
modelo = DecoderLM()

In [36]:
summary(modelo)

Layer (type:depth-idx)                        Param #
DecoderLM                                     --
├─TransformerDecoder: 1-1                     --
│    └─Embedding: 2-1                         1,488,000
│    └─PositionalEncoding: 2-2                --
│    │    └─Dropout: 3-1                      --
│    └─ModuleList: 2-3                        --
│    │    └─DecoderBlock: 3-2                 190,108
│    │    └─DecoderBlock: 3-3                 190,108
│    │    └─DecoderBlock: 3-4                 190,108
│    │    └─DecoderBlock: 3-5                 190,108
│    │    └─DecoderBlock: 3-6                 190,108
│    │    └─DecoderBlock: 3-7                 190,108
│    └─LayerNorm: 2-4                         248
├─Linear: 1-2                                 1,500,000
Total params: 4,128,896
Trainable params: 4,128,896
Non-trainable params: 0

In [37]:
modelo.to(device)

DecoderLM(
  (decoder): TransformerDecoder(
    (embedding): Embedding(12000, 124, padding_idx=0)
    (pos_encoding): PositionalEncoding(
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (decoder_blocks): ModuleList(
      (0-5): 6 x DecoderBlock(
        (attention): MultiHeadAttention(
          (out_projection): Linear(in_features=124, out_features=124, bias=True)
          (proj_q): Linear(in_features=124, out_features=124, bias=True)
          (proj_k): Linear(in_features=124, out_features=124, bias=True)
          (proj_v): Linear(in_features=124, out_features=124, bias=True)
          (dropout_attention): Dropout(p=0.1, inplace=False)
          (dropout_projection): Dropout(p=0.1, inplace=False)
        )
        (feedforward): FeedFowardBlock(
          (ff_1): Linear(in_features=124, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (ff_2): Linear(in_features=512, out_features=124, bias=True)
          (activation): NewGELU()
   

In [38]:
optimizer = torch.optim.AdamW(modelo.parameters(), lr=5e-5)

In [39]:
dataloader = DataLoader(
    train_dataset, num_workers=0, shuffle=True, batch_size=BATCH_SIZE
)

In [54]:
update_count = 0
accum_loss = None

optimizer.zero_grad(set_to_none=True)
for epoca in range(EPOCHS):
    batch_iterator = tqdm(dataloader, desc=f"Processing Epoch {epoch:02d}")
    for batch in batch_iterator:
        x, y = batch['decoder_input'], batch['true_output']
        x = x.to(device)
        y = y.long().to(device)

        pad_mask = (x != 0).type(torch.int).reshape((-1,))
        non_pad_indexes = torch.flatten(pad_mask.nonzero())

        # also flat the labels
        labels = y.reshape((-1,)).type(torch.long)
        labels = labels[non_pad_indexes]
        # runs through the model
        out = modelo(x, non_pad_indexes)
        loss = torch.nn.functional.cross_entropy(out, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)

        if accum_loss is not None:
            accum_loss = 0.99 * accum_loss + 0.01 * loss.detach().cpu().item()
        else:
            accum_loss = loss.detach().cpu().item()

        update_count += 1
        batch_iterator.set_description(
            f"Epoca: {epoca}\tUpdate: {update_count}\tLoss: {loss.detach().cpu().item():.4f}\tAccum_loss: {accum_loss:.4f}"
        )

Epoca: 0	Update: 2500	Loss: 4.6935	Accum_loss: 4.5727: 100%|███████████████████████| 2500/2500 [01:37<00:00, 25.70it/s]
Epoca: 1	Update: 5000	Loss: 4.5517	Accum_loss: 4.5292: 100%|███████████████████████| 2500/2500 [01:41<00:00, 24.52it/s]
Epoca: 2	Update: 7500	Loss: 4.6824	Accum_loss: 4.4645: 100%|███████████████████████| 2500/2500 [01:41<00:00, 24.63it/s]
Epoca: 3	Update: 10000	Loss: 4.6143	Accum_loss: 4.4545: 100%|██████████████████████| 2500/2500 [01:41<00:00, 24.55it/s]
Epoca: 4	Update: 12500	Loss: 4.4253	Accum_loss: 4.4196: 100%|██████████████████████| 2500/2500 [01:42<00:00, 24.30it/s]
Epoca: 5	Update: 15000	Loss: 4.3295	Accum_loss: 4.3856: 100%|██████████████████████| 2500/2500 [01:41<00:00, 24.67it/s]
Epoca: 6	Update: 17500	Loss: 4.4416	Accum_loss: 4.3616: 100%|██████████████████████| 2500/2500 [01:41<00:00, 24.66it/s]
Epoca: 7	Update: 20000	Loss: 4.1651	Accum_loss: 4.3385: 100%|██████████████████████| 2500/2500 [01:41<00:00, 24.70it/s]
Epoca: 8	Update: 22500	Loss: 4.5494	Accu

In [58]:
generation_pipeline = LMPipeline(
    mask_token_id=4, tokenizer=tokenizer, model=modelo, sos_token=1, eos_token=2
)

# input_text = """Eu só tenho uma coisa a dizer sobre este produto"""
input_text = input()
# input_text = 'o aplicativo'

texto_gerado = generation_pipeline.decoder_standard_generation(
    input_text=input_text,
    max_tokens=100,
    decoder_max_len=MAX_LEN,
    do_sample=True,
    temperature=0.8,
    top_k=20,
    num_breams=2,
    repetition_penalty=1.2,
    device=device,
)
print(texto_gerado)
print("*" * 50)

texto_gerado = generation_pipeline.decoder_nucleus_generation(
    input_text=input_text,
    max_tokens=100,
    decoder_max_len=MAX_LEN,
    p=0.95,
    temperature=0.8,
    repetition_penalty=1.2,
    device=device,
)

print(texto_gerado)

 nao gostei do aplicativo


 nao gostei do aplicativo  e bem mais nao foi que o app de boa minha conta no meu <unk> por isso e sinceramente ele deveria ter a opcao de editar como antes mas se fosse me comunicar com <unk> eu ja escolhia desta forma para o telegram pois uso desse aplicativo por um motivo tipo de conversas da pessoa que estava dando as mensagens antigas so tenho contatos amigos ou grupos a mensagem ela nunca mais achei pessimo muito chato sendo q era possivel criar uma amiga novas culturas entao porfavor do tinder faz essas coisas que ainda tem me ajudar nisso ta de
**************************************************
 nao gostei do aplicativo  e verdade que nao gostei fiz uma compra pelo item q entrega compra por falta de restaurantes ok mas o app esta com pagamento em dinheiro sou cancelar meu cartao de debito e recebi no mbway porque fiz um reembolso pago a resposta fornecedora e com o uber eats ja que nao tiveram suporte do aplicativo pra resolver fiz contato nunca mais de entrar pelo email hoje e